# for Meeting Chairpersons 2.0

Each Chairperson indicated the dates they are free.
This program allocates all Chairpersons to a unique timeslot each.

In [ ]:
import pandas as pd

### 1) Getting the Data

In [ ]:
data = pd.read_excel('meet.xlsx').fillna('NA')
data

In [ ]:
# dict of clubs' available dates
availability = {}

In [ ]:
# list of weeks
weeks = []
for i in range(2, 8):
    weeks.append(data.columns[i])
weeks

In [ ]:
for idx, row in data.iterrows():
    
    # each club will have a list of tuples indicating available dates
    availability[row["Club Name"]] = []
    
    for i in range(len(weeks)):
        # for weeks that have available day(s)
        if row[weeks[i]] != 'NA':
            
            # add tuple with format (week, day) into list
            days = row[weeks[i]].split(", ")
            for day in days:
                availability[row["Club Name"]].append((weeks[i], day))

In [ ]:
for club in availability:
    print(club, ":", sep="")
    print(availability[club])
    print()

### 2) Allocating the Timeslots

In [ ]:
# template name
template = data["Club Name"][0]

# dict of allocated dates for each club
allocated = {}

# number of clubs to allocate (-1 because 1 is template free dates)
total = len(availability) - 1

#### 2a) First-Come First-Serve Solution

In [ ]:
for club in availability:
    if club == template:
        continue

    # find a free date for the club
    for date in availability[club]:
        if date in availability[template]:
            allocated[club] = date
            availability[template].remove(date)
            break

    # flag out club without any compatible free dates
    if club not in allocated:
        print(club, "has no compatible free dates.")

#### 2b) Allocate clubs with less availabilities first Solution

In [ ]:
# sort clubs from least availabilities to most
for club in sorted(availability, key=lambda club: len(availability[club])):
    if club == template:
        continue

    # find a free date for the club
    for date in availability[club]:
        if date in availability[template]:
            allocated[club] = date
            availability[template].remove(date)
            break

    # flag out club without any compatible free dates
    if club not in allocated:
        print(club, "has no compatible free dates.")

#### 2c) Backtracking Solution

In [ ]:
def allocate():
    # base case: all clubs have been allocated
    if len(allocated) == len(availability) - 1:
        return True
    
    for club in availability:
        # skip clubs that are allocated a date already
        if club in allocated or club == template:
            continue
        
        # find a free date to allocate to the club
        for date in availability[club]:
            if date in availability[template]:
                
                # allocate that date to the club & remove from template free dates
                allocated[club] = date
                availability[template].remove(date)
                
                # if possible to allocate a date for the next club (and all other clubs), exit function
                if allocate():
                    return True
                
                # cannot find a date to allocate to next club
                else:
                    # backtrack: remove current club's allocation & update template free dates
                    allocated.pop(club)
                    availability[template].append(date)
                    # loop again to try next available date
        
        # no available dates remaining
        return False
    
    # all clubs have been allocated (this shouldn't run)
    return True
    
    
if allocate():
    print("Allocation done!")
else:
    print("Some error encountered :(")

### Allocation Results

In [ ]:
# print results
for club in allocated:
    print(club, ": ", allocated[club][1], ", ", allocated[club][0], sep="")

In [ ]:
# dataframe for better visualisation :D
pd.DataFrame(allocated, index=["Week", "Day"]).transpose()